# Combining Datasets: Concat and Append
Some of the most interesting studies of data come from combining different data
sources. These operations can involve anything from very straightforward concatenation
of two different datasets, to more complicated database-style joins and merges
that correctly handle any overlaps between the datasets. Series and DataFrames are
built with this type of operation in mind, and Pandas includes functions and methods
that make this sort of data wrangling fast and straightforward.

In [1]:
import numpy as np 
import pandas as pd 

For convenience, we’ll define this function, which creates a DataFrame of a particular
form that will be useful below:

In [5]:
def make_df(cols, ind):
    """Quickly make a DataFrame"""
    data = {c: [str(c) + str(i) for i in ind] for c in cols}
    return pd.DataFrame(data, ind)
# example DataFrame
make_df('ABC', range(3))

,A,B,C
0,A0,B0,C0
1,A1,B1,C1
2,A2,B2,C2


### Recall: Concatenation of NumPy Arrays
Concatenation of Series and DataFrame objects is very similar to concatenation of
NumPy arrays, which can be done via the np.concatenate function.m Recall that with it, you can combine the
contents of two or more arrays into a single array:

In [7]:
# array concatenation
x = [1, 2, 3]
y = [4, 5, 6]
z = [7, 8, 9]
np.concatenate([x, y, z])

array([1, 2, 3, 4, 5, 6, 7, 8, 9])

The first argument is a list or tuple of arrays to concatenate. Additionally, it takes an
axis keyword that allows you to specify the axis along which the result will be
concatenated:

In [9]:
x = [[1, 2], [3, 4]]
np.concatenate([x, x], axis = 1)

array([[1, 2, 1, 2],
       [3, 4, 3, 4]])

## Simple Concatenation with pd.concat
Pandas has a function, pd.concat(), which has a similar syntax to np.concatenate
but contains a number of options:

# Signature in Pandas v0.18
pd.concat(objs, axis=0, join='outer', join_axes=None, ignore_index=False, keys=None, levels=None, names=None,verify_integrity=False, copy=True)

pd.concat() can be used for a simple concatenation of Series or DataFrame objects,
just as np.concatenate() can be used for simple concatenations of arrays:

In [17]:
ser1 = pd.Series(['A', 'B', 'C'], index=[1, 2, 3])
ser2 = pd.Series(['D', 'E', 'F'], index=[4, 5, 6])
# pd.concat()
pd.concat([ser1, ser2])

1    A
2    B
3    C
4    D
5    E
6    F
dtype: object

It also works to concatenate higher-dimensional objects, such as DataFrames:

In [19]:
df1 = make_df('AB', [1, 2])
df2 = make_df('AB', [3, 4])
# conacatenting DataFrames
print(df1); print(df2); print(pd.concat([df1, df2]))

A   B
1  A1  B1
2  A2  B2
    A   B
3  A3  B3
4  A4  B4
    A   B
1  A1  B1
2  A2  B2
3  A3  B3
4  A4  B4


By default, the concatenation takes place row-wise within the DataFrame (i.e.,
axis=0). Like np.concatenate, pd.concat allows specification of an axis along which
concatenation will take place. Consider the following example:

In [22]:
df3 = make_df('AB', [0, 1])
df4 = make_df('CD', [0, 1])
# concatenation along columns
print(df3); print(df4); print(pd.concat([df3, df4], axis=1))

A   B
0  A0  B0
1  A1  B1
    C   D
0  C0  D0
1  C1  D1
    A   B   C   D
0  A0  B0  C0  D0
1  A1  B1  C1  D1


### Duplicate indices
One important difference between np.concatenate and pd.concat is that Pandas
concatenation preserves indices, even if the result will have duplicate indices! Consider
this simple example:

In [30]:
x = make_df('AB', [0, 1])
y = make_df('AB', [2, 3])
y.index = x.index # make duplicate indices!
print(x); print(y); print(pd.concat([x, y]))

A   B
0  A0  B0
1  A1  B1
    A   B
0  A2  B2
1  A3  B3
    A   B
0  A0  B0
1  A1  B1
0  A2  B2
1  A3  B3


Notice the repeated indices in the result. While this is valid within DataFrames, the
outcome is often undesirable. pd.concat() gives us a few ways to handle it.

<mark><b>Catching the repeats as an error</b></mark>. If you’d like to simply verify that the indices in the
result of pd.concat() do not overlap, you can specify the verify_integrity flag.
With this set to True, the concatenation will raise an exception if there are duplicate
indices. Here is an example, where for clarity we’ll catch and print the error message:

In [34]:
# Catching the repeats as an error
try:
    pd.concat([x, y], verify_integrity=True)
except ValueError as e:
    print("ValueError:", e)

ValueError: Indexes have overlapping values: Int64Index([0, 1], dtype='int64')



<mark><b>Ignoring the index</b></mark>. Sometimes the index itself does not matter, and you would prefer
it to simply be ignored. You can specify this option using the <mark><b>ignore_index</b></mark> flag. With
this set to True, the concatenation will create a new integer index for the resulting
Series:

In [33]:
# Ignoring Index
print(x); print(y); print(pd.concat([x, y], ignore_index=True))

A   B
0  A0  B0
1  A1  B1
    A   B
0  A2  B2
1  A3  B3
    A   B
0  A0  B0
1  A1  B1
2  A2  B2
3  A3  B3


<mark><b>Adding MultiIndex keys</b></mark> . Another alternative is to use the keys option to specify a label
for the data sources; the result will be a hierarchically indexed series containing the
data:

In [35]:
# Adding MultiIndex keys
print(x); print(y); print(pd.concat([x, y], keys=['x', 'y']))

A   B
0  A0  B0
1  A1  B1
    A   B
0  A2  B2
1  A3  B3
      A   B
x 0  A0  B0
  1  A1  B1
y 0  A2  B2
  1  A3  B3


The result is a multiply indexed DataFrame, and we can use the tools discussed in
“Hierarchical Indexing” to transform this data into the representation
we’re interested in.

## Concatenation with joins
In the simple examples we just looked at, we were mainly concatenating DataFrames
with shared column names. In practice, data from different sources might have different
sets of column names, and pd.concat offers several options in this case. Consider
the concatenation of the following two DataFrames, which have some (but not all!)
columns in common:

In [41]:
df5 = make_df('ABC', [1, 2])
df6 = make_df('BCD', [3, 4])

In [42]:
df5

,A,B,C
1,A1,B1,C1
2,A2,B2,C2


In [43]:
df6

,B,C,D
3,B3,C3,D3
4,B4,C4,D4


In [44]:
pd.concat([df5, df6])

,A,B,C,D
1,A1,B1,C1,NaN
2,A2,B2,C2,NaN
3,NaN,B3,C3,D3
4,NaN,B4,C4,D4


By default, the entries for which no data is available are filled with NA values. To
change this, we can specify one of several options for the join and <b>join_axes</b> parameters
of the concatenate function. By default, the join is a union of the input columns
(join='outer'), but we can change this to an intersection of the columns using
join='inner':

In [46]:
# inner join
pd.concat([df5, df6], join='inner')

,B,C
1,B1,C1
2,B2,C2
3,B3,C3
4,B4,C4


Another option is to directly specify the index of the remaining colums using the
<b>join_axes</b> argument, which takes a list of index objects. Here we’ll specify that the
returned columns should be the same as those of the first input:

In [47]:
pd.concat([df5, df6], join_axes=[df5.columns])

,A,B,C
1,A1,B1,C1
2,A2,B2,C2
3,NaN,B3,C3
4,NaN,B4,C4


The combination of options of the pd.concat function allows a wide range of possible
behaviors when you are joining two datasets;

### The append() method
Because direct array concatenation is so common, Series and DataFrame objects
have an append method that can accomplish the same thing in fewer keystrokes. For
example, rather than calling <b>pd.concat([df1, df2])</b>, you can simply call
<mark><b>df1.append(df2)</b></mark>:

In [50]:
# the append() method
df1.append(df2)

,A,B
1,A1,B1
2,A2,B2
3,A3,B3
4,A4,B4


Keep in mind that unlike the append() and extend() methods of Python lists, the
append() method in Pandas does not modify the original object—instead, it creates a
new object with the combined data. It also is not a very efficient method, because it
involves creation of a new index and data buffer. Thus, if you plan to do multiple
append operations, it is generally better to build a list of DataFrames and pass them all
at once to the concat() function.